### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта 

## ОТЧЕТ ПО КОНТРОЛЬНОЙ РАБОТЕ № 5


### Дисциплина: Методы машинного обучения

## Москва 2024
***

##### Студент:  Артамонов Т.Е.
##### Группа:   НКНбд-01-21

Контрольная работа 5 Вариант 14 

1. Набор данных stl10 
2. Классы с метками 4,5,6 
3. Требования к архитектуре сети MLP: 
Кол-во скрытых слоев 5 
Кол-во нейронов 50 в каждом нечетном скрытом слое, 70 в каждом четном скрытом слое 
Оптимизатор Adamax 
Функция активации в скрытых слоях elu 
Регуляризация L1 в каждом нечетном скрытом слое 
4. Требования к архитектуре сети CNN: 
Кол-во слоев пулинга 4 
Количество фильтров в сверточных слоях 8 
Размеры фильтра 5х5 
Оптимизатор Adafactor 
Функция активации в сверточных слоях tanh 
Функция активации в скрытых плотных слоях selu 
Регуляризация L2 в каждом скрытом плотном слое 
5. Показатель качества бинарной классификации: 
индекс Жаккара, равный TP/(TP + FP + FN) 
6. Показатель качества многоклассовой классификации: 
средняя точность классов, где точность (precision) класса равна доле правильных предсказаний для всех точек, относимых классификатором к этому классу. 

### Задание 1. Загрузите заданный в индивидуальном задании набор данных с изображениями из Tensorflow Datasets с разбиением на обучающую и тестовую выборки.

### Решение
Импортируем библиотеки

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from PIL import Image, ImageOps

Загрузим тренировочную и тестовую части датасета

In [ ]:
ds = tfds.load("stl10", split=['train', 'test'])
df_train = tfds.as_dataframe(ds[0])
df_test = tfds.as_dataframe(ds[1])
df_train.head(3)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
df_train.shape, df_test.shape

### Задание 2.  Визуализируйте несколько изображений, отобранных случайным образом из обучающей выборки.

### Решение

Выведем несколько случайных изображений из датасета

In [ ]:
import random

def plot_random_sample(images):
    n = 10
    imgs = random.sample(list(images), n)

    num_row = 2
    num_col = 5

    fig, axes = plt.subplots(num_row, num_col, figsize=(3.5 * num_col, 3 * num_row))
    # For every image
    for i in range(num_row * num_col):
        # Read the image
        img = imgs[i]
        # Display the image
        ax = axes[i // num_col, i % num_col]
        ax.imshow(img)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_random_sample(df_train['image'])

### Задание 3. Оставьте в наборе изображения двух классов, указанных в индивидуальном задании первыми. Обучите нейронные сети MLP и CNN задаче бинарной классификации изображений (требования к архитектуре сетей указаны в индивидуальном задании). Отследите обучение нейронных сетей и укажите, на сколько процентов снизились в результате обучения потери по отношению к потерям на первой эпохе обучения. Оцените результаты обучения нейронных сетей (варианты: нейронная сеть обучилась, недообучилась, переобучилась).

### Решение

Выберем из датасета только изображения с метками 4 и 5, заменим эти метки на 0 и 1

In [ ]:
x = df_train[df_train['label'] == 4]
x['label'] = 0
y = df_train[df_train['label'] == 5]
y['label'] = 1
df_tr1 = pd.concat([x, y])


x = df_test[df_test['label'] == 4]
x['label'] = 0
y = df_test[df_test['label']  == 5]
y['label'] = 1

df_te1 = pd.concat([x, y])

df_tr1['label'].value_counts()


In [ ]:
df_te1['label'].value_counts()

Изображения 96х96 пикселей с rgb

In [ ]:
df_train.iloc[0]['image'].shape

Проверим, что все распределилось правильно

In [ ]:
plot_random_sample(df_te1['image'])

In [ ]:
plot_random_sample(df_tr1['image'])

В классах 4 и 5 олени и собаки

Предобработаем данные из датасета

In [ ]:
train_labels = df_tr1['label'].to_numpy(dtype=np.float32)
test_labels = df_te1['label'].to_numpy(dtype=np.float32)
train_images = np.zeros(shape=(df_tr1.shape[0],96,96,3), dtype=np.float32)
test_images  = np.zeros(shape=(df_te1.shape[0],96,96,3), dtype=np.float32)

for idx in range(train_labels.shape[0]):
    train_images[idx,:,:,:] = \
        np.array(Image.fromarray(df_tr1.iloc[idx]['image']))

for idx in range(test_labels.shape[0]):
    test_images[idx,:,:,:] = \
        np.array(Image.fromarray(df_te1.iloc[idx]['image']))
    
train_images /= 255
test_images  /= 255

train_images.shape, test_images.shape

Создадим модель MLP и обучим ее соответственно заданию

In [ ]:
tf.random.set_seed(42)

model_1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(96, 96, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50, activation='elu', kernel_regularizer='l1'),
    tf.keras.layers.Dense(70, activation='elu'),
    tf.keras.layers.Dense(50, activation='elu', kernel_regularizer='l1'),
    tf.keras.layers.Dense(70, activation='elu'),
    tf.keras.layers.Dense(50, activation='elu', kernel_regularizer='l1'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_1.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adamax(0.00008),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

history_1 = model_1.fit(
    train_images,
    train_labels,
    epochs=100,
    batch_size=128,
    validation_data=(test_images, test_labels)
)

Модель переобучилась, точность упала

In [ ]:
tf.random.set_seed(42)

model_2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),

    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(128, activation='selu', kernel_regularizer='l2'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_2.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adafactor(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

history_2 = model_2.fit(
    train_images,
    train_labels,
    epochs=100,
    batch_size=128,
    validation_data=(test_images, test_labels)
)

Модель переобучилась, разница между потерями в тренировочной и тестовой выборке увеличивается

In [ ]:
print("Снижение потерь модели MLP на",(1 - history_1.history['loss'][-1] / history_1.history['loss'][0])*100,'%')
print("Снижение потерь модели CNN на", (1 - history_2.history['loss'][-1] / history_2.history['loss'][0])*100,'%')

### Задание 4. Постройте кривые обучения нейронных сетей бинарной классификации для показателей потерь и доли верных ответов в зависимости от эпохи обучения, подписывая оси и рисунок и создавая легенду. 

### Решение
Построим для первой модели 2 разных графика для потерь и доли верных ответов, т.к. размеры графиков сильно отличаются

In [ ]:
plt.plot(np.arange(1, 101), history_1.history['loss'], label='Training Loss')
plt.plot(np.arange(1, 101), history_1.history['val_loss'], label='Validation Loss')
plt.title('Training vs. Validation Loss', size=20)
plt.xlabel('Epoch', size=14)
plt.ylabel('Loss', size=14)
plt.legend();

In [ ]:
plt.plot(np.arange(1, 101), history_1.history['accuracy'], label='Accuracy')
plt.plot(np.arange(1, 101), history_1.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs. Validation Accuracy', size=20)
plt.xlabel('Epoch', size=14)
plt.ylabel('Accuracy', size=14)
plt.legend();

Видно, что точность после 70 эпохи начинает падать

Построим график для второй модели

In [ ]:
plt.plot(np.arange(1, 101), history_2.history['loss'], label='Training Loss')
plt.plot(np.arange(1, 101), history_2.history['val_loss'], label='Validation Loss')
plt.plot(np.arange(1, 101), history_2.history['accuracy'], label='Accuracy')
plt.plot(np.arange(1, 101), history_2.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs. Validation loss and accuracy', size=20)
plt.xlabel('Epoch', size=14)
plt.ylabel('Loss', size=14)
plt.legend();

Видно, что линии функций тренировочных и валидационных данных начинают расходиться, это свидетельствует о переобучении

### Задание 5. Сравните качество бинарной классификации нейронными сетями при помощи показателя качества, указанного в индивидуальном задании.

### Решение

In [ ]:
def jk(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    tp = np.sum((y_true == 0) & (y_predict == 0))
    fp = np.sum((y_true == 0) & (y_predict == 1))
    fn = np.sum((y_true == 1) & (y_predict == 0))
    return tp/(tp + fn + fp)

In [ ]:
y1 = np.array([1 if prob > 0.5 else 0 for prob in np.ravel(model_1.predict(train_images))])
y2 = np.array([1 if prob > 0.5 else 0 for prob in np.ravel(model_2.predict(train_images))])
print("Индекс Жаккара для первой модели:", jk(y1, np.array(train_labels))) 
print("Индекс Жаккара для второй модели:", jk(y2, np.array(train_labels)))

Показатель второй модели гораздо выше

### Задание 6. Визуализируйте ROC-кривые для построенных классификаторов на одном рисунке (с легендой) и вычислите площади под ROC-кривыми.

### Решение

In [ ]:
def true_false_positive(threshold_vector, y_test):
    true_positive = np.equal(threshold_vector, 1) & np.equal(y_test, 1)
    true_negative = np.equal(threshold_vector, 0) & np.equal(y_test, 0)
    false_positive = np.equal(threshold_vector, 1) & np.equal(y_test, 0)
    false_negative = np.equal(threshold_vector, 0) & np.equal(y_test, 1)

    tpr = true_positive.sum() / (true_positive.sum() + false_negative.sum())
    fpr = false_positive.sum() / (false_positive.sum() + true_negative.sum())

    return tpr, fpr
def roc_from_scratch(probabilities, y_test, partitions=100):
    roc = np.array([])
    for i in range(partitions + 1):

        threshold_vector = np.greater_equal(probabilities, i / partitions).astype(int)
        tpr, fpr = true_false_positive(threshold_vector, y_test)
        roc = np.append(roc, [fpr, tpr])

    return roc.reshape(-1, 2)

In [ ]:
from sklearn.metrics import roc_auc_score
plt.figure(figsize=(15,7))
m1 = model_1.predict(train_images)
m2 = model_2.predict(train_images)
r1 = roc_auc_score(train_labels, m1)
r2 = roc_auc_score(train_labels, m2)
ROC1 = roc_from_scratch(m1.reshape(-1),train_labels,partitions=50)
plt.plot(ROC1[:,0],ROC1[:,1],lw=5, label='Model 1, ROC AUC=' + str(r1))

ROC2 = roc_from_scratch(m2.reshape(-1),train_labels,partitions=50)
plt.plot(ROC2[:,0],ROC2[:,1],lw=5, label='Model 2, ROC AUC=' + str(r2))

plt.title('ROC кривая',fontsize=20)
plt.xlabel('Показатель FPR (False Positive Rate)',fontsize=16)
plt.ylabel('Показатель TPR (True Positive Rate)',fontsize=16)
plt.legend();

Площадь под кривой второй модели лучше

### Задание 7. Оставьте в наборе изображения трех классов, указанных в индивидуальном задании. Обучите нейронные сети MLP и CNN задаче многоклассовой классификации изображений (требования к архитектуре сетей указаны в индивидуальном задании).

### Решение
Оставим в наборе изображения с метками 4, 5, 6, переименуем их в 0, 1, 2

In [ ]:
x = df_train[df_train['label'] == 4]
x['label'] = 0
y = df_train[df_train['label'] == 5]
y['label'] = 1
z = df_train[df_train['label'] == 6]
z['label'] = 2

df_tr2 = pd.concat([x, y, z])

x = df_test[df_test['label'] == 4]
x['label'] = 0
y = df_test[df_test['label']  == 5]
y['label'] = 1
z = df_test[df_test['label']  == 6]
z['label'] = 2

df_te2 = pd.concat([x, y, z])

df_tr2['label'].value_counts(), df_te2['label'].value_counts()

Применим one-hot encoding для меток, предобработаем данные

In [ ]:
Y1 = list(df_tr2['label'])
for i in range(len(Y1)):
    tmp = [0]*3
    tmp[Y1[i]] = 1
    Y1[i] = tmp

Y2 = list(df_te2['label'])
for i in range(len(Y2)):
    tmp = [0]*3
    tmp[Y2[i]] = 1
    Y2[i] = tmp

train_labels = np.array(Y1, dtype=np.float32)
test_labels = np.array(Y2, dtype=np.float32)


train_images = np.zeros(shape=(df_tr2.shape[0],96,96,3), dtype=np.float32)
test_images  = np.zeros(shape=(df_te2.shape[0],96,96,3), dtype=np.float32)

for idx in range(train_labels.shape[0]):
    train_images[idx,:,:,:] = \
        np.array(Image.fromarray(df_tr2.iloc[idx]['image']))

for idx in range(test_labels.shape[0]):
    test_images[idx,:,:,:] = \
        np.array(Image.fromarray(df_te2.iloc[idx]['image']))
    
train_images /= 255
test_images  /= 255

train_images.shape, test_images.shape, train_labels.shape, test_labels.shape

Создадим и обучим модель MLP

In [ ]:
tf.random.set_seed(42)

model_3 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(96, 96, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50, activation='elu', kernel_regularizer='l1'),
    tf.keras.layers.Dense(70, activation='elu'),
    tf.keras.layers.Dense(50, activation='elu', kernel_regularizer='l1'),
    tf.keras.layers.Dense(70, activation='elu'),
    tf.keras.layers.Dense(50, activation='elu', kernel_regularizer='l1'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model_3.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adamax(0.00005),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy')]
)

history_3 = model_3.fit(
    train_images,
    train_labels,
    epochs=100,
    batch_size=128,
    validation_data=(test_images, test_labels)
)

In [ ]:
tf.random.set_seed(42)

model_4 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),

    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5),
                           activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(128, activation='selu', kernel_regularizer='l2'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model_4.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adafactor(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy')]
)

history_4 = model_4.fit(
    train_images,
    train_labels,
    epochs=100,
    batch_size=128,
    validation_data=(test_images, test_labels)
)

### 8. Сравните качество многоклассовой классификации нейронными сетями при помощи показателя качества, указанного в индивидуальном задании.

### Решение

In [ ]:
m = tf.keras.metrics.Precision()
m.update_state(train_labels, model_3.predict(train_images))
x1 = m.result().numpy()
m.reset_state()
m.update_state(train_labels, model_4.predict(train_images))
x2 = m.result().numpy()
print("Precision первой модели:", x1) 
print("Precision второй модели:", x2) 

У первой модели показатель качества выше

### Задание 9. Постройте кривые обучения нейронных сетей многоклассовой классификации для показателей ошибки и доли верных ответов в зависимости от эпохи обучения, подписывая оси и рисунок и создавая легенду. Сопроводите программный код необходимыми комментариями.

### Решение
Для первой модели два графика, т.к. размеры отличаются

In [ ]:
plt.plot(np.arange(1, 101), history_3.history['loss'], label='Training Loss')
plt.plot(np.arange(1, 101), history_3.history['val_loss'], label='Validation Loss')

plt.title('Model 1 Training vs. Validation loss', size=18)
plt.xlabel('Epoch', size=14)
plt.ylabel('Loss', size=14)
plt.legend();

Если судить по показателю потерь, то все хорошо

In [ ]:
plt.plot(np.arange(1, 101), history_3.history['accuracy'], label='Accuracy')
plt.plot(np.arange(1, 101), history_3.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 1 Training vs. Validation accuracy', size=18)
plt.xlabel('Epoch', size=14)
plt.ylabel('Accuracy', size=14)
plt.legend();

Но показатели метрики падают, модель переобучилась

In [ ]:
plt.plot(np.arange(1, 101), history_4.history['loss'], label='Training Loss')
plt.plot(np.arange(1, 101), history_4.history['val_loss'], label='Validation Loss')
plt.plot(np.arange(1, 101), history_4.history['accuracy'], label='Accuracy')
plt.plot(np.arange(1, 101), history_4.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 2 Training vs. Validation loss and accuracy', size=18)
plt.xlabel('Epoch', size=14)
plt.legend();

Видно, что постепенно разница между Training и Validation растёт, модель переобучилась.